# Includes

In [1]:
#include <math.h>
#include <iostream>

In [2]:
#pragma cling add_include_path("/Users/benjamin/repositories/awi-workspace/Bias-Adjustment-Cpp/include/") // change for the specific location of the header files
#pragma cling add_include_path("/Users/benjamin/repositories/awi-workspace/Bias-Adjustment-Cpp/src/") // change for the specific location of the source files
#pragma cling add_library_path("/usr/local/lib")
#pragma cling add_include_path("/usr/local/include")

In [3]:
#include <netcdf>
#pragma cling load("netcdf-cxx4")

In [4]:
#pragma cling load("CMethods.hxx")
#pragma cling load("NcFileHandler.hxx")
#pragma cling load("MyMath.hxx")
#pragma cling load("Utils.hxx")
#pragma cling load("colors.h")

#pragma cling load("CMethods.cxx")
#pragma cling load("NcFileHandler.cxx")
#pragma cling load("MyMath.cxx")
#pragma cling load("Utils.cxx")

____
# Setup

In [5]:
std::string variable_name = "tas";

In [6]:
NcFileHandler
    ds_obs = NcFileHandler("input_data/obs.nc", variable_name),
    ds_simh = NcFileHandler("input_data/contr.nc", variable_name),
    ds_simp = NcFileHandler("input_data/scen.nc", variable_name);

In [7]:
float* obs_one_loc = new float[ds_obs.n_time]; 
float* simh_one_loc = new float[ds_simh.n_time];
float* simp_one_loc = new float[ds_simp.n_time];

float* ls_result = new float[ds_simp.n_time];
float* qdm_result = new float[ds_simp.n_time];

____
# Select grid box

In [8]:
// select time series
ds_obs.fill_timeseries_for_location(obs_one_loc, 0, 0);
ds_simh.fill_timeseries_for_location(simh_one_loc, 0, 0);
ds_simp.fill_timeseries_for_location(simp_one_loc, 0, 0);

# Select methods

In [9]:
// select metotds 
CM_Func_ptr_simple apply_adjustment_ls = CMethods::get_cmethod_simple("linear_scaling");
CM_Func_ptr_quantile apply_adjustment_qdm = CMethods::get_cmethod_quantile("quantile_delta_mapping");

# Apply methods

In [10]:
// apply linear scaling
apply_adjustment_ls(
    ls_result, 
    obs_one_loc, 
    simh_one_loc, 
    simp_one_loc, 
    ds_simp.n_time,
    "+"
)

In [11]:
// apply quantile delta mapping
apply_adjustment_qdm(
    qdm_result, 
    obs_one_loc, 
    simh_one_loc, 
    simp_one_loc, 
    ds_simp.n_time,
    "+",
    100
)

In [12]:
int start = 0, end = 10;

# Observations

In [13]:
for(unsigned ts = start; ts < end; ts++) std::cout << obs_one_loc[ts] << " ";

-24.0942 -24.417 -24.1754 -24.0361 -23.7633 -24.1706 -23.71 -24.5677 -24.654 -23.4294 

# Model (control)

In [14]:
for(unsigned ts = start; ts < end; ts++) std::cout << simh_one_loc[ts] << " ";

-26.0942 -26.417 -26.1754 -26.0361 -25.7633 -26.1706 -25.71 -26.5677 -26.654 -25.4294 

# Model (scenario)

In [15]:
for(unsigned ts = start; ts < end; ts++) std::cout << simp_one_loc[ts] << " ";

-25.0942 -25.417 -25.1754 -25.0361 -24.7633 -25.1706 -24.71 -25.5677 -25.654 -24.4294 

# LS-adjusted
Note: this must be applied to monthly data! So this is incomplete.

In [16]:
for(unsigned ts = 0; ts < 10; ts++) std::cout << ls_result[ts] << " ";

-23.0942 -23.417 -23.1754 -23.0361 -22.7633 -23.1706 -22.71 -23.5677 -23.654 -22.4294 

# QDM-adjusted

In [17]:
for(unsigned ts = 0; ts < 10; ts++) std::cout << qdm_result[ts] << " ";

-23.1014 -23.4159 -23.1805 -23.047 -22.7724 -23.1758 -22.716 -23.5665 -23.6526 -22.4402 

In [18]:
// save to file 
ds_simp.save_to_netcdf("qdm_result.nc", variable_name, qdm_result, ds_simp.n_time)
/** Note: 
 *    the last parameter is n_time, so you can save datasets that does not match the NcFileHandlers` (<ds_simp>) dimensions 
 */